In [2]:
# I used colab for this part, to use the additional Ram that they provide, to be able to run demanding queries.
# I even used an advanced GPU to utilise the extra Ram that came with it.
# This way some calculations finished in 10% of the time. Others only became posssible this way.
import polars as pl
from datetime import timedelta, datetime
import math

In [2]:
# with scan, a file can be read into a lazy format. it takes considerably less time and ram.
parq = pl.scan_parquet("/content/drive/MyDrive/Taxitaxi/Combined_Parquet/Polars/*")

In [ ]:
# with read, a usual dataframe is being created. requires more ressoures.
# to counter that, only the desired columns can be selected and read
columns='PULocationID'
parq = pl.read_parquet("/content/drive/MyDrive/Taxitaxi/Combined_Parquet/Polars/*",columns=columns)


In [3]:
# the given date format is has the format dd/mm/yyyy/HH/MM/SS, which increases the size.
#in the following line, new columns for the single parts replace the actual date column
#this allows easier filtering and reduces the data size

parq=parq.with_columns(pl.col("tpep_pickup_datetime").dt.year().alias("Year"))
parq=parq.with_columns(pl.col("tpep_pickup_datetime").dt.weekday().alias("Weekday"))
parq=parq.with_columns(pl.col("tpep_pickup_datetime").dt.month().alias("Month"))
parq=parq.with_columns(pl.col("tpep_pickup_datetime").dt.day().alias("Day"))
parq=parq.with_columns(pl.col("tpep_pickup_datetime").dt.hour().alias("Hour"))

#afterwards, the needed columns can be kept and the rest dropped
columns=['PULocationID',"Year","trip_distance","total_amount"]
parq=parq.select(pl.col(columns))
parq.columns

['PULocationID', 'Year', 'trip_distance', 'total_amount']

#In the following, I created a code for every type of groupby+aggregation I needed for a specific insight

In [7]:

tripsperyear=(parq
 .select(
    pl.col("Year","PULocationID"))
 .groupby("PULocationID","Year")
 .agg([pl.col("Year").count().alias("Number of Trips")])
  .collect()
)


In [4]:
tripdistanceperyear=(parq
 .select(
    pl.col("Year","trip_distance","PULocationID"))
 .groupby("Year")
 .agg([pl.col("PULocationID").count().alias("Number of Trips"),pl.col("trip_distance").sum().alias("Trip_distance")])
  .collect()
)

In [5]:
tripsperhour=(parq
 .select(
    pl.col("Hour","PULocationID"))
 .groupby("PULocationID","Hour")
 .agg([pl.col("Hour").count().alias("Number of Trips")])
  .collect()
)


In [ ]:
revenue_monthly=(parq
 .select(
    pl.col("total_amount","Month","PULocationID"))
 .groupby("Month","PULocationID")
 .agg([pl.col("total_amount").mean().alias("Mean Revenue")])
  .collect()
)

In [7]:
revenuepertrip=(parq
 .select(
    pl.col("Year","PULocationID","total_amount"))
 .groupby("PULocationID")
 .agg([pl.col("Hour").count().alias("Number of Trips"),pl.col("total_amount").sum().alias("Revenue")])
  .collect()
)


In [4]:
revenueperyear=(parq
 .select(
    pl.col("Year","Day","PULocationID","total_amount"))
 .groupby("PULocationID","Year")
 .agg([pl.col("Day").count().alias("Number of Trips"),pl.col("total_amount").sum().alias("Revenue")])
  .collect()
)

In [ ]:
#have a glimpse of the output without loading it completely
tripdistanceperyear.glimpse

In [5]:
#saving the output to the online drive
tripdistanceperyear.write_csv(r"/content/drive/MyDrive/Taxitaxi/Tableau/tripdistanceperyear.csv")